In [1]:
# Intialization
import os
import time

import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

# data science imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
# import unicodedata

In [2]:
# Set Locations
data_path = 'C:/Users/IOLAP-USER/Documents/Movie-Recommendation-System/Data/Reviews-1M/'

In [3]:
# Import one-hot encoded movies metadata
movies_df = pd.read_csv(data_path + 'movies_metadata_ohe.csv')
movies_df['itemId'] = movies_df.itemId.apply(lambda x: 'itemId_' + str(x))
movies_df = movies_df.set_index(movies_df.itemId) # set index so no sorting errors occur

# Import ratings data
ratings_df = pd.read_csv(data_path + 'ratings.dat',
                        sep = '::', header = None)
ratings_df.columns = ['userId', 'itemId', 'label', 'timestamp']
ratings_df.drop(['timestamp'], axis = 1, inplace = True)
ratings_df['userId'] = ratings_df.userId.apply(lambda x: 'userId_' + str(x))
ratings_df['itemId'] = ratings_df.itemId.apply(lambda x: 'itemId_' + str(x))

# Import users data
users_df = pd.read_csv(data_path + 'users.dat', 
                       sep = '::', header = None)
users_df.columns = ['userId', 'gender', 'age_group', 'occupation', 'zipcode']
users_df.drop(['zipcode'], axis = 1, inplace = True)
users_df['userId'] = users_df.userId.apply(lambda x: 'userId_' + str(x))

C:\Users\IOLAP-USER\Anaconda3\envs\py37env\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\IOLAP-USER\Anaconda3\envs\py37env\lib\site-packages\ipykernel_launcher.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [4]:
movies_df.head(5)

,itemId,title,imdb_id,imdb_rating,imdb_votes,metascore,runtime,year,ml_genre_Action,ml_genre_Adventure,...,MPAA_rating_PG,MPAA_rating_PG13,MPAA_rating_R,MPAA_rating_TV14,MPAA_rating_TVG,MPAA_rating_TVMA,MPAA_rating_TVPG,MPAA_rating_Unrated,MPAA_rating_X,MPAA_rating_Other
itemId,,,,,,,,,,,,,,,,,,,,,
itemId_3699,itemId_3699,Starman,tt0088172,7.0,38393,71,115,1984,0,1,...,1,0,0,0,0,0,0,0,0,0
itemId_3700,itemId_3700,"Brother from Another Planet, The",tt0087004,6.8,5050,0,108,1984,0,0,...,0,0,1,0,0,0,0,0,0,0
itemId_3701,itemId_3701,Alien Nation,tt0094631,6.3,12867,45,91,1988,0,0,...,0,0,1,0,0,0,0,0,0,0
itemId_3702,itemId_3702,Mad Max,tt0079501,7.0,173984,73,88,1979,1,0,...,0,0,1,0,0,0,0,0,0,0
itemId_3703,itemId_3703,Mad Max 2 (a.k.a. The Road Warrior),tt0082694,7.6,152866,77,94,1981,1,0,...,0,0,1,0,0,0,0,0,0,0


In [5]:
users_df.head(5)

,userId,gender,age_group,occupation
0,userId_1,F,1,10
1,userId_2,M,56,16
2,userId_3,M,25,15
3,userId_4,M,45,7
4,userId_5,M,25,20


In [6]:
ratings_df.head(5)

,userId,itemId,label
0,userId_1,itemId_1193,5
1,userId_1,itemId_661,3
2,userId_1,itemId_914,3
3,userId_1,itemId_3408,4
4,userId_1,itemId_2355,5


Find the unique set of movies that have ratings in ratings_df. Remove all movies without ratings from movies_df.
There are 3883 movies total, and only 3706 movies with ratings.

In [7]:
print(len(ratings_df.itemId.unique()))
#3706 < 3883 so some movies do not have any ratings, these movies can be removed for genre rating averaging
# keep only movies that have at least one user rating
distinct_rated_movies = ratings_df.itemId.unique()
movies_df_rated = movies_df.iloc[[item in distinct_rated_movies for item in movies_df.itemId]]
print(movies_df_rated.shape)

3706
(3706, 10214)


Transform the ratings dataframe (userId, itemId, label) to a userId x itemId dataframe with the labels (ratings) as the values. Fill all missing values, items that have not been rated by a user, with 0s. 

Then create the binarized version of this matrix, and again fill any missing values with 0s. This dataframe will have values of 1 where a rating exists, and 0 otherwise.

In [8]:
# Transform ratings dataframe to user_id x item_id dataframe with label (or rating) as the values
ratings_spread = ratings_df.pivot(index='userId',columns='itemId',values='label').fillna(0)
# Create binary valued dataframe from the transformed ratings dataframe. 
#   This should have values of 1 where the labels are >0 and 0 otherwise.
ratings_spread_binary = (ratings_spread/ratings_spread).fillna(0)

print('User Movie Ratings')
ratings_spread.head()

User Movie Ratings


itemId,itemId_1,itemId_10,itemId_100,itemId_1000,itemId_1002,itemId_1003,itemId_1004,itemId_1005,itemId_1006,itemId_1007,...,itemId_99,itemId_990,itemId_991,itemId_992,itemId_993,itemId_994,itemId_996,itemId_997,itemId_998,itemId_999
userId,,,,,,,,,,,,,,,,,,,,,
userId_1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
userId_10,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
userId_100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
userId_1000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
userId_1001,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Movie Genre Rating Summary

In [9]:
# Create movies dataframe subset that only contains the Movie Lens genre or IMDb genre.
genre_cols = [mg for mg in list(movies_df_rated.columns) if (mg[0:8] == 'ml_genre' or mg[0:10] == 'imdb_genre')]
movie_genres = movies_df_rated[genre_cols]
# Cross the new ratings dataframe with the genres dataframe 
#   - this is now a datframe with sum of ratings by genre for each user
user_genre_total = ratings_spread.dot(movie_genres)
# Cross the binarized ratings dataframe with the genres dataframe 
#   - this is now a dataframe that contains counts of rated movies by genre for each user
user_genre_count = ratings_spread_binary.dot(movie_genres)

In [10]:
ratings_spread_binary.head()

itemId,itemId_1,itemId_10,itemId_100,itemId_1000,itemId_1002,itemId_1003,itemId_1004,itemId_1005,itemId_1006,itemId_1007,...,itemId_99,itemId_990,itemId_991,itemId_992,itemId_993,itemId_994,itemId_996,itemId_997,itemId_998,itemId_999
userId,,,,,,,,,,,,,,,,,,,,,
userId_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
userId_10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
userId_100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
userId_1000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
userId_1001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
movie_genres.head()

,ml_genre_Action,ml_genre_Adventure,ml_genre_Animation,ml_genre_Childrens,ml_genre_Comedy,ml_genre_Crime,ml_genre_Documentary,ml_genre_Drama,ml_genre_Fantasy,ml_genre_FilmNoir,...,imdb_genre_Mystery,imdb_genre_RealityTV,imdb_genre_Romance,imdb_genre_SciFi,imdb_genre_Short,imdb_genre_Sport,imdb_genre_Thriller,imdb_genre_War,imdb_genre_Western,imdb_genre_nan
itemId,,,,,,,,,,,,,,,,,,,,,
itemId_3699,0,1,0,0,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
itemId_3700,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
itemId_3701,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
itemId_3702,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
itemId_3703,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [12]:
# Perform element wise division on user genre total and user genre count to get user genre average
user_genre_avg = user_genre_total / user_genre_count
user_genre_avg = user_genre_avg.add_suffix('_avg_rating').reset_index()

users_metadata = pd.merge(users_df, user_genre_avg, on = 'userId')

print('User Metadata with Genre Averages')
users_metadata.head()

User Metadata with Genre Averages


,userId,gender,age_group,occupation,ml_genre_Action_avg_rating,ml_genre_Adventure_avg_rating,ml_genre_Animation_avg_rating,ml_genre_Childrens_avg_rating,ml_genre_Comedy_avg_rating,ml_genre_Crime_avg_rating,...,imdb_genre_Mystery_avg_rating,imdb_genre_RealityTV_avg_rating,imdb_genre_Romance_avg_rating,imdb_genre_SciFi_avg_rating,imdb_genre_Short_avg_rating,imdb_genre_Sport_avg_rating,imdb_genre_Thriller_avg_rating,imdb_genre_War_avg_rating,imdb_genre_Western_avg_rating,imdb_genre_nan_avg_rating
0,userId_1,F,1,10,4.200000,4.000000,4.111111,4.250000,4.142857,4.000000,...,4.000000,NaN,4.062500,4.333333,3.0,NaN,4.000000,4.500000,NaN,NaN
1,userId_2,M,56,16,3.500000,3.736842,NaN,NaN,3.560000,3.583333,...,3.400000,NaN,3.692308,3.555556,4.0,4.6,3.456522,3.538462,4.000000,NaN
2,userId_3,M,25,15,3.956522,4.000000,4.000000,4.000000,3.766667,NaN,...,3.500000,NaN,3.800000,4.000000,NaN,4.5,4.000000,NaN,4.571429,NaN
3,userId_4,M,45,7,4.157895,3.833333,NaN,4.000000,NaN,5.000000,...,NaN,NaN,NaN,3.600000,NaN,5.0,4.000000,5.000000,4.500000,NaN
4,userId_5,M,25,20,2.612903,3.000000,4.000000,3.833333,3.410714,3.285714,...,3.294118,NaN,3.133333,3.000000,3.0,NaN,2.711111,3.571429,3.500000,3.0


In [13]:
del movie_genres, user_genre_total, user_genre_count, user_genre_avg

## Actor and Director Ratings Summary

In [14]:
# Create movies dataframe subset that only contains the actors or directors
actor_cols = [mg for mg in list(movies_df_rated.columns) if mg[0:6] == 'actor_']
director_cols = [mg for mg in list(movies_df_rated.columns) if mg[0:9] == 'director_']
people_cols = actor_cols + director_cols
movie_people = movies_df_rated[people_cols]
# # Cross the binarized ratings dataframe with the genres dataframe 
# #   - this is now a dataframe that contains counts of rated movies by genre for each user
user_movie_people_count = ratings_spread_binary.dot(movie_people)

In [15]:
# For testing purposes, create list of Alfred Hitchcock movies (can be done for any director)
hitchcock_movies = list(movies_df_rated.itemId.iloc[np.where(movies_df_rated.director_Alfred_Hitchcock == 1)])
print(hitchcock_movies)

['itemId_1086', 'itemId_1219', 'itemId_903', 'itemId_904', 'itemId_908', 'itemId_928', 'itemId_929', 'itemId_930', 'itemId_931', 'itemId_933', 'itemId_965', 'itemId_1333', 'itemId_2176', 'itemId_2177', 'itemId_2178', 'itemId_2179', 'itemId_2180', 'itemId_2181', 'itemId_2182', 'itemId_2183', 'itemId_2184', 'itemId_2185', 'itemId_2186', 'itemId_2187', 'itemId_2200', 'itemId_2201', 'itemId_2202', 'itemId_2203', 'itemId_2204', 'itemId_2205', 'itemId_2206', 'itemId_2207', 'itemId_2208', 'itemId_2209', 'itemId_2210', 'itemId_2211', 'itemId_2212', 'itemId_2213', 'itemId_2214', 'itemId_2215', 'itemId_2217', 'itemId_2218', 'itemId_2219', 'itemId_2221', 'itemId_2223', 'itemId_2226', 'itemId_2227']


In [16]:
# Determine how many ratings were made for Hitchcock movies
ratings_df_temp = ratings_df.iloc[np.where([item in hitchcock_movies for item in ratings_df.itemId])]
print('number of ratings:', len(ratings_df_temp))

number of ratings: 10247


In [17]:
# Make sure that the ratings were spread correctly and that the number of ratings remains constant
print('number of ratings:', np.sum(np.sum(ratings_spread_binary[hitchcock_movies])))

number of ratings: 10247.0


In [18]:
# Confirm that the dot product maintained a correct number of ratings
print('number of ratings:', np.sum(user_movie_people_count.director_Alfred_Hitchcock))

number of ratings: 10247.0


In [19]:
actor_views = user_movie_people_count[actor_cols].apply(np.sum, axis = 0)
# This total will be much greater than the total number of ratings because an individual movie has multiple actors
print('total actor views: ', np.sum(actor_views))
director_views = user_movie_people_count[director_cols].apply(np.sum, axis = 0)
# This total may be greater than the total number of ratings if one movie had multiple directors
print('total director views: ', np.sum(director_views))

total actor views:  3983239.0
total director views:  1117394.0


In [20]:
del movie_people, user_movie_people_count, hitchcock_movies, ratings_df_temp

In [21]:
# Filter actors to top 200, or ~7800, based on number of views
top_200_actors = actor_views.sort_values(ascending = False).head(200)
top_200_actors_names = top_200_actors.index
top_200_actors

actor_Harrison_Ford            24807.0
actor_Tom_Hanks                18570.0
actor_Robert_De_Niro           16254.0
actor_Mel_Gibson               14855.0
actor_Bruce_Willis             14146.0
actor_Sean_Connery             13722.0
actor_Arnold_Schwarzenegger    13501.0
actor_Gene_Hackman             13254.0
actor_Sigourney_Weaver         12528.0
actor_Tom_Cruise               11736.0
actor_Jack_Nicholson           11420.0
actor_Kevin_Spacey             10881.0
actor_Al_Pacino                10862.0
actor_Carrie_Fisher            10806.0
actor_Bill_Murray               9844.0
actor_Nicolas_Cage              9827.0
actor_John_Cusack               9792.0
actor_Robin_Williams            9783.0
actor_John_Travolta             9435.0
actor_Christopher_Lloyd         9318.0
actor_Jeff_Goldblum             9041.0
actor_Mark_Hamill               8864.0
actor_Dan_Aykroyd               8722.0
actor_Tommy_Lee_Jones           8670.0
actor_Kevin_Costner             8518.0
actor_Dustin_Hoffman     

In [22]:
# Filter directors to top 50, of ~2200, based on number of views
top_50_directors = director_views.sort_values(ascending = False).head(50)
top_50_directors_names = top_50_directors.index
top_50_directors

director_Steven_Spielberg        21170.0
director_Robert_Zemeckis         12411.0
director_James_Cameron           11228.0
director_Rob_Reiner              10278.0
director_Alfred_Hitchcock        10247.0
director_Richard_Donner          10099.0
director_Tim_Burton               9358.0
director_Stanley_Kubrick          8786.0
director_Ridley_Scott             8454.0
director_Francis_Ford_Coppola     8244.0
director_John_McTiernan           8042.0
director_Ron_Howard               7921.0
director_Woody_Allen              7884.0
director_Barry_Levinson           7168.0
director_Joel_Coen                6629.0
director_Ethan_Coen               6629.0
director_Martin_Scorsese          6568.0
director_Terry_Gilliam            6276.0
director_George_Lucas             6231.0
director_Barry_Sonnenfeld         6040.0
director_Paul_Verhoeven           5761.0
director_Wilfred_Jackson          5592.0
director_Ivan_Reitman             5526.0
director_Wolfgang_Petersen        5494.0
director_John_La

In [23]:
# Filter out actors and directors not in the top 200 and 50
# Create movies dataframe subset that only contains the top actors and directors
movie_people = movies_df_rated[list(top_200_actors_names) + list(top_50_directors_names)]
# Cross the new ratings dataframe with the  movie_people dataframes
#   - this is now a datframe with sum of ratings by actor and director for each user
user_movie_people_total = ratings_spread.dot(movie_people)
# Cross the binarized ratings dataframe with the movie_people dataframes
#   - this is now a dataframe with counts of rated movies by actor and director for each user
user_movie_people_count = ratings_spread_binary.dot(movie_people)

In [24]:
user_movie_people_avg_ratings = user_movie_people_total / user_movie_people_count
user_movie_people_avg_ratings = user_movie_people_avg_ratings.add_suffix('_avg_rating').reset_index()

In [25]:
users_metadata = pd.merge(users_metadata, user_movie_people_avg_ratings, on = 'userId')
gender_OHE = pd.get_dummies(users_metadata.gender, prefix = 'gender')
age_group_OHE = pd.get_dummies(users_metadata.age_group, prefix = 'age_group')
users_metadata = pd.concat([users_metadata, gender_OHE, age_group_OHE], axis = 1, sort = False)
users_metadata.drop(['gender', 'age_group', 'occupation'], axis = 1, inplace = True)
users_metadata.head()

,userId,ml_genre_Action_avg_rating,ml_genre_Adventure_avg_rating,ml_genre_Animation_avg_rating,ml_genre_Childrens_avg_rating,ml_genre_Comedy_avg_rating,ml_genre_Crime_avg_rating,ml_genre_Documentary_avg_rating,ml_genre_Drama_avg_rating,ml_genre_Fantasy_avg_rating,...,director_Robert_Altman_avg_rating,gender_F,gender_M,age_group_1,age_group_18,age_group_25,age_group_35,age_group_45,age_group_50,age_group_56
0,userId_1,4.200000,4.000000,4.111111,4.250000,4.142857,4.000000,NaN,4.428571,4.0,...,NaN,1,0,1,0,0,0,0,0,0
1,userId_2,3.500000,3.736842,NaN,NaN,3.560000,3.583333,NaN,3.898734,3.0,...,NaN,0,1,0,0,0,0,0,0,1
2,userId_3,3.956522,4.000000,4.000000,4.000000,3.766667,NaN,NaN,4.000000,4.5,...,NaN,0,1,0,0,1,0,0,0,0
3,userId_4,4.157895,3.833333,NaN,4.000000,NaN,5.000000,NaN,4.166667,4.5,...,NaN,0,1,0,0,0,0,1,0,0
4,userId_5,2.612903,3.000000,4.000000,3.833333,3.410714,3.285714,3.666667,3.096154,NaN,...,4.0,0,1,0,0,1,0,0,0,0


In [26]:
del movie_people, user_movie_people_total, user_movie_people_count, \
    user_movie_people_avg_ratings, movies_df_rated, ratings_spread, ratings_spread_binary, \
    top_200_actors, top_50_directors, ratings_df, actor_views, \
    director_views, users_df

## Convert to Spark Dataframe and Save

In [27]:
users_metadata['userId'] = users_metadata.userId.apply(lambda x: int(x[7:]))
sorted_columns = list(users_metadata.columns.sort_values())
users_metadata = users_metadata[sorted_columns]
users_metadata.to_csv(data_path + 'users_metadata.csv', index = False)

In [28]:
movie_cols_to_keep = ['itemId', 'title', 'imdb_id', 'imdb_rating', 
                       'imdb_votes', 'metascore', 'runtime', 'year'] +\
        genre_cols + list(top_50_directors_names) + list(top_200_actors_names)
movie_metadata_ohe_subset = movies_df[movie_cols_to_keep]
movie_metadata_ohe_subset.to_csv(data_path + 'movie_metadata_ohe_subset.csv', index = False)

In [29]:
genre_and_people_cols = ['itemId'] + genre_cols + list(top_50_directors_names) + list(top_200_actors_names)
movie_genre_and_people_metadata_ohe_subset = movies_df[genre_and_people_cols]
movie_genre_and_people_metadata_ohe_subset.to_csv(data_path + 'movie_genre_and_people_metadata_ohe_subset.csv', index = False)